<a href="https://colab.research.google.com/github/Pruthviraj141/Colab-file/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.34.0 accelerate==0.23.0 bitsandbytes==0.41.1 peft==0.6.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2

Loading Model from Hugging face

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Step 1: Load the Falcon chatbot model
model_name = "heliosbrahma/falcon-7b-sharded-bf16-finetuned-mental-health-conversational"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

chatbot_tokenizer = AutoTokenizer.from_pretrained(model_name)
chatbot_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

# Step 2: Load a pre-trained emotion detection model with quantization
emotion_model_name = "j-hartmann/emotion-english-distilroberta-base"

emotion_quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # NormalFloat4 for better accuracy
    bnb_4bit_compute_dtype="float16"  # Mixed precision for computation
)

emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)
emotion_model = AutoModelForSequenceClassification.from_pretrained(
    emotion_model_name,
    quantization_config=emotion_quantization_config,
    device_map="auto"  # Automatically map layers to available devices
)

print("Emotion detection model loaded with quantization!")

emotion_pipeline = pipeline("text-classification", model=emotion_model, tokenizer=emotion_tokenizer)



/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

Emotion detection model loaded with quantization!


Getting responce by User Input through Code

In [ ]:
# Function to detect emotion
def detect_emotion(user_input):
    result = emotion_pipeline(user_input)
    emotion = result[0]["label"]
    confidence = result[0]["score"]
    return emotion, confidence

# Function to truncate conversation history to the last few exchanges
def truncate_conversation(conversation, max_length=500):
    # Ensure the conversation history does not exceed max_length
    if len(conversation) > max_length:
        return conversation[-max_length:]  # Keep only the most recent characters
    return conversation

# Function to generate a response from the model with emotion adjustment
def generate_response_with_emotion(user_input):
    global conversation_history

    # Detect emotion
    emotion, confidence = detect_emotion(user_input)
    print(f"Detected Emotion: {emotion} (Confidence: {confidence:.2f})")

    # Append the user's input to the conversation history
    conversation_history += f"Human: {user_input}\nAI:"

    # Truncate the conversation history to keep it manageable
    conversation_history = truncate_conversation(conversation_history)

    # Tokenize the conversation history for AI generation
    inputs = chatbot_tokenizer(conversation_history, return_tensors="pt", truncation=True, max_length=1024).to(chatbot_model.device)
    outputs = chatbot_model.generate(
        inputs['input_ids'],
        max_length=150,
        num_return_sequences=1,
        pad_token_id=chatbot_tokenizer.eos_token_id,
        temperature=0.7,  # Added for more creative but controlled responses
        top_k=50
    )

    # Decode the AI's raw response and extract its part
    response = chatbot_tokenizer.decode(outputs[0], skip_special_tokens=True).split("AI:")[-1].strip()
    conversation_history += response + "\n"

    # Adjust the response tone based on detected emotion
    if emotion == "sadness":
        if confidence >= 0.95:
            print("If you need someone to talk to, call anytime: 1800-120-820050")
        response = f"I'm deeply sorry you're feeling this way. {response}"
    elif emotion == "joy":
        response = f"That's wonderful news! {response}"
    elif emotion == "anger":
        response = f"I understand how frustrating this must be. {response}"
    elif emotion == "fear":
        response = f"It’s understandable to feel this way. {response}"
    elif emotion == "love":
        response = f"That’s so beautiful to hear. {response}"

    # Ensure the final response is concise and empathetic
    return response.strip()

# Simulate multiple input-response cycles
conversation_history = ""  # Initialize or reset the conversation history

# First input
user_input_1 = "Today I had a breakup with my girlfriend, so I don't wanna live this life anymore."
response_1 = generate_response_with_emotion(user_input_1)
print("AI's Response:", response_1)

# Second input
user_input_2 = "I feel a bit better now, but I'm still worried about the future."
response_2 = generate_response_with_emotion(user_input_2)
print("AI's Response:", response_2)


Detected Emotion: sadness (Confidence: 0.97)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` an

If you need someone to talk to, call anytime: 1800-120-820050
AI's Response: I'm deeply sorry you're feeling this way. I'm really sorry to hear that you're feeling so hopeless. It's completely understandable to feel this way after such a difficult experience. Your girlfriend's decision to break up with you has probably left you feeling rejected and abandoned.

Remember, I'm here to listen and support you. Can you tell me a bit more about what's been going on or what challenges you've faced? It's essential to talk about feelings like this to process them and find ways to heal.

I'm here to provide you with encouragement and guidance, but I encourage you to seek professional
Detected Emotion: fear (Confidence: 0.32)
AI's Response: It’s understandable to feel this way. I'm glad to hear that you're feeling a bit better, but it's completely understandable to feel worried about the future.


another way for getting output

In [ ]:
# Function to detect emotion
def detect_emotion(user_input):
    result = emotion_pipeline(user_input)
    emotion = result[0]["label"]
    confidence = result[0]["score"]
    return emotion, confidence

# Function to truncate conversation history to the last few exchanges
def truncate_conversation(conversation, max_length=500):
    # Ensure the conversation history does not exceed max_length
    if len(conversation) > max_length:
        return conversation[-max_length:]  # Keep only the most recent characters
    return conversation

# Function to generate a response from the model with emotion adjustment
def generate_response_with_emotion(user_input):
    global conversation_history

    # Detect emotion
    emotion, confidence = detect_emotion(user_input)
    print(f"Detected Emotion: {emotion} (Confidence: {confidence:.2f})")

    # Append the user's input to the conversation history
    conversation_history += f"Human: {user_input}\nAI:"

    # Truncate the conversation history to keep it manageable
    conversation_history = truncate_conversation(conversation_history)

    # Tokenize the conversation history for AI generation
    inputs = chatbot_tokenizer(conversation_history, return_tensors="pt", truncation=True, max_length=1024).to(chatbot_model.device)
    outputs = chatbot_model.generate(
        inputs['input_ids'],
        max_length=150,
        num_return_sequences=1,
        pad_token_id=chatbot_tokenizer.eos_token_id,
        temperature=0.7,  # Added for more creative but controlled responses
        top_k=50
    )

    # Decode the AI's raw response and extract its part
    response = chatbot_tokenizer.decode(outputs[0], skip_special_tokens=True).split("AI:")[-1].strip()
    conversation_history += response + "\n"

    # Adjust the response tone based on detected emotion
    if emotion == "sadness":
        if confidence >= 0.95:
            print("If you need someone to talk to, call anytime: 1800-120-820050")
        response = f"I'm deeply sorry you're feeling this way. {response}"
    elif emotion == "joy":
        response = f"That's wonderful news! {response}"
    elif emotion == "anger":
        response = f"I understand how frustrating this must be. {response}"
    elif emotion == "fear":
        response = f"It’s understandable to feel this way. {response}"
    elif emotion == "love":
        response = f"That’s so beautiful to hear. {response}"

    # Ensure the final response is concise and empathetic
    return response.strip()

# Simulate multiple input-response cycles
conversation_history = ""  # Initialize or reset the conversation history

# First input
user_input_1 = "Today I had a breakup with my girlfriend, so I don't wanna live this life anymore."
response_1 = generate_response_with_emotion(user_input_1)
print("AI's Response:", response_1)

# Second input
user_input_2 = "I feel a bit better now, but I'm still worried about the future."
response_2 = generate_response_with_emotion(user_input_2)
print("AI's Response:", response_2)


Detected Emotion: sadness (Confidence: 0.97)
If you need someone to talk to, call anytime: 1800-120-820050
AI's Response: I'm deeply sorry you're feeling this way. I'm really sorry to hear that you're feeling so hopeless. It's completely understandable to feel this way after such a difficult experience. Your girlfriend's decision to break up with you has probably left you feeling rejected and abandoned.

Remember, I'm here to listen and support you. Can you tell me a bit more about what's been going on or what challenges you've faced? It's essential to talk about feelings like this to process them and find ways to heal.

I'm here to provide you with encouragement and guidance, but I also encourage you to consider
Detected Emotion: fear (Confidence: 0.32)
AI's Response: It’s understandable to feel this way. I'm glad to hear that you're feeling a bit better, but it's completely understandable to feel worried about the future.


Trying to meantain the seassion with User ( needed to be more efficient )

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize conversation history
conversation_history = ""

# Functions for conversation and emotion detection
def detect_emotion(user_input):
    result = emotion_pipeline(user_input)
    emotion = result[0]["label"]
    confidence = result[0]["score"]
    return emotion, confidence

def truncate_conversation(conversation, max_length=500):
    if len(conversation) > max_length:
        return conversation[-max_length:]
    return conversation

def generate_response_with_emotion(user_input):
    global conversation_history

    # Detect emotion
    emotion, confidence = detect_emotion(user_input)

    # Update conversation history
    conversation_history += f"Human: {user_input}\nAI:"

    # Truncate history
    conversation_history = truncate_conversation(conversation_history)

    # Generate AI response
    inputs = chatbot_tokenizer(conversation_history, return_tensors="pt", truncation=True, max_length=1024).to(chatbot_model.device)
    outputs = chatbot_model.generate(
        inputs['input_ids'],
        max_length=200,
        num_return_sequences=1,
        pad_token_id=chatbot_tokenizer.eos_token_id,
        temperature=0.7,
        top_k=50
    )

    response = chatbot_tokenizer.decode(outputs[0], skip_special_tokens=True).split("AI:")[-1].strip()
    conversation_history += response + "\n"

    # Adjust response based on emotion
    if emotion == "sadness" and confidence >= 0.95:
        response = f"{confidence}, you can talk to someone anytime: 1800-120-820050\n{response}"
    elif emotion == "joy":
        response = f"Joy : {confidence} + {response}"
    elif emotion == "anger":
        response = f"Anger : {confidence}. + {response}"
    elif emotion == "fear":
        response = f"Fear : {confidence}. + {response}"
    elif emotion == "love":
        response = f"love {confidence}.  +  {response}"

    return response.strip()

# UI components
input_box = widgets.Text(
    placeholder='Enter your message...',
    description='You:',
    layout=widgets.Layout(width='90%')
)
output_box = widgets.Output(layout=widgets.Layout(width='90%', height='300px', overflow='auto'))
send_button = widgets.Button(description="Send")
reset_button = widgets.Button(description="Reset Session", button_style='danger')

# Handlers
def send_message(_):
    user_input = input_box.value
    if user_input.strip():
        with output_box:
            print(f"You: {user_input}")
        response = generate_response_with_emotion(user_input)
        with output_box:
            print(f"AI: {response}")
    input_box.value = ""

def reset_session(_):
    global conversation_history
    conversation_history = ""
    with output_box:
        clear_output()
        print("Session reset. Start a new conversation!")

# Bind actions
send_button.on_click(send_message)
reset_button.on_click(reset_session)

# Layout
ui = widgets.VBox([
    output_box,
    widgets.HBox([input_box, send_button]),
    reset_button
])

# Display the UI
display(ui)

# Initialize message
with output_box:
    print("Welcome to the chatbot! How can I assist you today?")


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
